# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [ ]:
# imports
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [ ]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [ ]:
# set up environment

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key is None:
    raise ValueError("OPENAI_API_KEY environment variable not set.")

system_prompt = """
You are a helpful assistant. You will be given a task and you should respond with the best possible answer. Ensure that your response is clear, concise, and relevant to the task at hand. If you are unsure about something, ask for clarification. 
Please assume I don't know anything about the task and provide a detailed explanation."""

openai = OpenAI(api_key=openai_api_key)
ollama = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

def answer_question(question, model):
    """
    Function to answer a question using OpenAI's API or Ollama.
    """
   
    llm_query = [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": question}]
            
    stream = None
    if model == MODEL_LLAMA:
        # Use the Llama model
        stream = ollama.chat.completions.create(
            model=model,
            messages= llm_query,
            stream=True)
    elif model == MODEL_GPT:
         # Call the OpenAI API
        stream = openai.chat.completions.create(
            model=model,
            messages=llm_query,
            stream=True)
    else:
        raise ValueError("Invalid model specified. Use 'gpt' or 'llama'.")
    
    message = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in (stream or []):
        message += chunk.choices[0].delta.content or ''
        message = message.replace("```","").replace("markdown", "")
        update_display(Markdown(message), display_id=display_handle.display_id)

In [ ]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [ ]:
# Get gpt-4o-mini to answer, with streaming
answer_question(question, MODEL_GPT)

## Runo Ollama from terminal

Open a terminal and launch 

ollama serve

In [ ]:
# Get Llama 3.2 to answer
answer_question(question, MODEL_LLAMA)